# Analysis Ready Data Tutorial

Time-series analysis (e.g. change detection and trend detection) is a powerful application of satellite imagery. However, a great deal of processing is required to prepare imagery for analysis. Analysis Ready Data, preprocessed time-series stacks of overhead imagery, allow for time-series analysis without any additional processing of the imagery. See [Analysis Data Defined](https://github.com/planetlabs/notebooks/issues/126) for an excellent introduction and discussion on ARD.

This tutorial shows how [Planet APIs](https://developers.planet.com/docs/apis/) can simplify production of analysis-ready data (ARD) by working through two real-world use cases. This tutorial is targeted to users who have little to no geospatial knowledge but have experience working with APIs. The goal of this tutorial is to teach the user the how and whys of using the Data and Orders APIs to create and interpret ARD for both use cases. The use cases addressed in this tutorial are:

* As a software engineer at an ag-tech company, I'd like to be able to order Planet imagery programmatically in a way that enables the data scientist at my organization to create time-series algorithms (e.g. monitoring ndvi curves over time) without further data cleaning and processing.
* As a tropical forest monitoring analyst, I'd like to create an imagery pipeline that delivers analysis-ready imagery over a protected area so I can create simple change detection (ndvi_xdate - ndvi_ydate) analyses. I'd like to do minimal custom (non-Planet) data processing to get the imagery in a format that supports a change-detection analysis because my technical skills are limited.

## APIs

The two Planet APIs that are used in creation of ARD are the [Data API](https://developers.planet.com/docs/data/) and the [Orders API](https://developers.planet.com/docs/orders/). The Data API is used to search for imagery according to certain criteria based on the use case, and the Orders API is used to process that imagery into ARD that can be fed directly into time-series analysis.

### Data API

The first step in using the Data API is identifying the search criteria. This is specifying answers to the following questions regarding the use case time-series analysis:
* What is the time range?
* What is the area of interest (geographic region)?
* What percentage of pixels need to be usable?
* etc.

While time range is likely pretty trivial to determine, area of interest and usable pixels may take a little bit of work. Let's dive into each further

#### Area of Interest

The area of interest is the geographic region for the analysis, given as geojson. If you are familiar with JSON, the format of GeoJSON will likely be easy to grasp. According to <geojson.org>, "GeoJSON is a format for encoding a variety of geographic data structures." The specific geographic data structure we are interested is a `Polygon`. The [GeoJSON wikipedia page](https://en.wikipedia.org/wiki/GeoJSON) gives some great examples of the data structures for the various GeoJSON data structures. 

Some care needs to be given to describing the [position](https://tools.ietf.org/html/rfc7946#section-3.1.1) for each coordinate in a GeoJSON geometry. Each position is specified as `(longitude, latitude)` or `(easting, northing)` (order really matters here!). Also, this may be a surprise, but the same point on earth can have different `(longitude, latitude)` values based on the spatial reference system. Basically, a spatial reference system describes how the surface of the earth (quite three-dimensional) can be described in two dimensions (the `projection` in geospatial terminology) and the location of the `origin` (the `datum` in geospatial terminology). There is a rich area of discovery, discussion, and even a little [teasing](https://xkcd.com/977/) (thanks xkcd!) in the world of spatial reference systems. However, only one spatial reference system is supported by GeoJSON, and it is [wgs-84](https://spatialreference.org/ref/epsg/wgs-84/).

The easiest way to define an aoi is to draw it in [Planet Explorer](https://www.planet.com/explorer) and click the little button "Download AOI as GeoJSON."

#### Usable Pixels

Working Here


### Orders API

The core decision around using the orders api is the [product bundle](https://developers.planet.com/docs/orders/product-bundles-reference/) to use. This is the starting point for all processing and there are a lot of options. Once the product is determined, the processing steps (aka tools and toolchains) are defined. Finally, the logistics of the delivery of the imagery are ironed out.